In [1]:
import omop_alchemy as oa
from omop_alchemy.db import Base, engine
from omop_alchemy.helpers.create_db import create_db 
from omop_alchemy.helpers.populate_db import populate_demo_db, to_load_vocabulary, to_load_clinical, to_load_health_system

In [2]:
# this script will create a new sqlite db file with the tables to reflect the specified objects in 
# clinical and reference directories. needs to be made more friendly for re-create / update etc, but 
# will do for now :)

create_db(Base, engine)

2024-02-07 09:53:39,347 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-07 09:53:39,348 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("modifiable_table")
2024-02-07 09:53:39,349 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-07 09:53:39,351 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("condition_occurrence")
2024-02-07 09:53:39,351 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-07 09:53:39,352 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("device_exposure")
2024-02-07 09:53:39,353 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-07 09:53:39,354 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("drug_exposure")
2024-02-07 09:53:39,354 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-07 09:53:39,356 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fact_relationship")
2024-02-07 09:53:39,357 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-07 09:53:39,357 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("measurement")
2024-02-07 09:53

In [3]:
# make sure to download a small athena zip file (suggested no more than 2-4 vocabs for demo purposes)
# and unzip to omop_alchemy/resources/ohdsi_vocabs before running this script. 

populate_demo_db(to_load_health_system)

/Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/demo_data
complete load for: CARE_SITE.csv
complete load for: LOCATION.csv
complete load for: PROVIDER.csv
2024-02-07 09:53:40,557 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-07 09:53:40,561 INFO sqlalchemy.engine.Engine INSERT INTO location (location_id, address_1, address_2, city, state, zip_code, county, country, location_source_value, latitude, longitude) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2024-02-07 09:53:40,563 INFO sqlalchemy.engine.Engine [generated in 0.00262s] [(1, 'Liverpool Hospital', 'Burnside Dr', 'Sydney', 'NSW', '2170', 'Liverpool', 'Australia', '', 0.0, 0.0), (2, 'Bankstown Lidcombe Hospital', 'Eldridge Rd', 'Sydney', 'NSW', '2200', 'Bankstown', 'Australia', '', 0.0, 0.0), (3, 'Campbelltown Hospital', 'Off Parkside Cres', 'Sydney', 'NSW', '2560', 'Campbelltown', 'Australia', '', 0.0, 0.0)]
2024-02-07 09:53:40,569 INFO sqlalchemy.engine.Engine INSERT INTO care_site (care_sit

/Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/helpers/populate_db.py:91: SAWarning: relationship 'Measurement.modified_object' will copy column modifiable_table.modifier_id to column measurement.modifier_of_event_id, which conflicts with relationship(s): 'Measurement.modifying_object' (copies modifiable_table.modifier_id to measurement.modifier_of_event_id), 'Modifiable_Table.modifiers' (copies modifiable_table.modifier_id to measurement.modifier_of_event_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="modifiers,modifying_object"' to the 'Measurement.modified_object' relationship. (Background on this wa

In [4]:

populate_demo_db(to_load_clinical)

/Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/demo_data
complete load for: PERSON.csv
complete load for: CONDITION_OCCURRENCE.csv
complete load for: MEASUREMENT.csv
2024-02-07 09:53:45,640 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-07 09:53:45,642 INFO sqlalchemy.engine.Engine INSERT INTO person (person_id, birth_datetime, death_datetime, person_source_value, gender_source_value, race_source_value, ethnicity_source_value, year_of_birth, month_of_birth, day_of_birth, location_id, provider_id, care_site_id, gender_source_concept_id, ethnicity_source_concept_id, race_source_concept_id, gender_concept_id, ethnicity_concept_id, race_concept_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2024-02-07 09:53:45,643 INFO sqlalchemy.engine.Engine [generated in 0.00088s] [(1, None, None, '1234', 'M', '', '', 1950, 1, 10, 1, 1, 1, 0, 0, 0, 8507, 0, 0), (2, None, None, '2354', 'F', '', '', 1960, 2, 11, 1, 2, 1, 0, 0, 0, 8532, 0, 0), 

In [ ]:
populate_demo_db(to_load_vocabulary)

In [5]:

import sqlalchemy.orm as so
import sqlalchemy.sql.sqltypes as sss
from omop_alchemy.db.config import engine, config
from omop_alchemy.tables.clinical import Person, Condition_Occurrence

with so.Session(engine) as sess:
    people = sess.query(Person
                        ).options(so.joinedload(Person.conditions, 
                                                Condition_Occurrence.condition_concept)).all()

2024-02-07 09:53:49,522 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-07 09:53:49,526 INFO sqlalchemy.engine.Engine SELECT person.person_id AS person_person_id, person.birth_datetime AS person_birth_datetime, person.death_datetime AS person_death_datetime, person.person_source_value AS person_person_source_value, person.gender_source_value AS person_gender_source_value, person.race_source_value AS person_race_source_value, person.ethnicity_source_value AS person_ethnicity_source_value, person.year_of_birth AS person_year_of_birth, person.month_of_birth AS person_month_of_birth, person.day_of_birth AS person_day_of_birth, person.location_id AS person_location_id, person.provider_id AS person_provider_id, person.care_site_id AS person_care_site_id, person.gender_source_concept_id AS person_gender_source_concept_id, person.ethnicity_source_concept_id AS person_ethnicity_source_concept_id, person.race_source_concept_id AS person_race_source_concept_id, person.gender_concept_id AS 

In [6]:
people

[Person: person_id = 1,
 Person: person_id = 2,
 Person: person_id = 3,
 Person: person_id = 4]

In [ ]:
import csv
from pathlib import Path
from datetime import datetime

import sqlalchemy as sa
import sqlalchemy.orm as so
import sqlalchemy.sql.sqltypes as sss

from omop_alchemy.db.config import engine, config
from omop_alchemy.tables.clinical import Person, Condition_Occurrence, Measurement
from omop_alchemy.tables.health_system import Care_Site, Location, Provider
from omop_alchemy.tables.vocabulary import Concept, Vocabulary, Concept_Class, Domain, Relationship, \
                                Concept_Relationship, Concept_Ancestor

# TODO: insert some validation and checks to make sure folk know how and why to do this (and the limits for demo purposes)

to_load_vocabulary = {'folder': 'ohdsi_vocabs',
                      'VOCABULARY.csv': Vocabulary, 
                      'CONCEPT.csv': Concept, 
                      'CONCEPT_CLASS.csv': Concept_Class, 
                      'DOMAIN.csv': Domain,
                      'RELATIONSHIP.csv': Relationship}
                      #'CONCEPT_RELATIONSHIP.csv': Concept_Relationship,
                      #'CONCEPT_ANCESTOR.csv': Concept_Ancestor}

to_load_health_system = {'folder': 'demo_data',
                         'CARE_SITE.csv': Care_Site,
                         'LOCATION.csv': Location,
                         'PROVIDER.csv': Provider}

to_load_clinical = {'folder': 'demo_data',
                    'PERSON.csv': Person,
                    'CONDITION_OCCURRENCE.csv': Condition_Occurrence,
                    'MEASUREMENT.csv': Measurement}#,
                    #'CONCEPT.csv': Concept}

# flexible loading of ohdsi vocab files downloaded to the path /data/ohdsi_vocabs

def datetime_conversion(dt, fmt):
    if dt != '':
        return datetime.strptime(dt, fmt)
    
def convert_date_col(dt):
    return datetime_conversion(dt, '%Y%m%d')
    
def convert_time_col(dt):
    return datetime_conversion(dt, '%H%M%S')

def convert_datetime_col(dt):
    return datetime_conversion(dt, '%Y%m%d%H%M%S')

def callable_pass(s):
    return s

def convert_int(i):
    try:
        return int(i)
    except:
        return 0
    
def convert_dec(i):
    try:
        return Decimal(i)
    except:
        return 0

type_map = {sss.BigInteger: convert_int, 
            sss.Integer: convert_int, 
            sss.Numeric: convert_dec, 
            sss.DateTime: convert_datetime_col, 
            sss.Time: convert_time_col, 
            sss.String: callable_pass, 
            sss.Date: convert_date_col}

def get_type_lookup(interface):
    return {c.key: type_map[type(c.type)] for c in interface.__table__._columns}



to_load = to_load_clinical

with so.Session(engine) as sess:
    folder = Path(config.VOCAB_PATH) / to_load['folder']
    print(folder)
    for ohdsi_file, interface in to_load.items():
        if interface != folder.name:
                with open(folder / ohdsi_file, 'r') as file:
                    reader = csv.DictReader(file, delimiter='\t')
                    field_map = get_type_lookup(interface)
                    
                    for row in reader:
                        record = {field:field_map[field](data) for field, data in row.items() if field in field_map}
                        o = interface(**record)
                        sess.add(o)

                    print(f'complete load for: {ohdsi_file}')
            
    sess.commit()
